In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:08<00:00,  3.68it/s]


eval images shape: (30, 1041, 1302) contain 10094 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.4,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 115.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 878.728 || Time Upd.: 408.8 ms  || BatchNr.: 100
 || Cost: 180.138 || Time Upd.: 384.6 ms  || BatchNr.: 200
 || Cost: 109.740 || Time Upd.: 383.1 ms  || BatchNr.: 300
 || Cost: 95.026 || Time Upd.: 381.7 ms  || BatchNr.: 400
 || Cost: 90.052 || Time Upd.: 384.8 ms  || BatchNr.: 500
 || Cost: 74.554 || Time Upd.: 388.9 ms  || BatchNr.: 600
 || Cost: 64.700 || Time Upd.: 379.5 ms  || BatchNr.: 700
 || Cost: 63.580 || Time Upd.: 381.0 ms  || BatchNr.: 800
 || Cost: 62.515 || Time Upd.: 382.1 ms  || BatchNr.: 900
 || Cost: 58.354 || Time Upd.: 391.3 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:5003
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 5003
JoR: 0.820 || Eff_3d: 9.487 || Jaccard: 0.495 || Factor: 277.958 || RMSE_lat: 60.429 || RMSE_ax: 177.962 || RM

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:6011
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 6011
JoR: 1.114 || Eff_3d: 33.564 || Jaccard: 0.595 || Factor: 283.599 || RMSE_lat: 53.417 || RMSE_ax: 103.825 || RMSE_x: 37.985 || RMSE_y: 37.557 || Cost: 45.792 || Recall: 0.595 || Precision: 1.000 || BatchNr.: 2300 || Cost: 45.792 || Time Upd.: 390.7 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 33.564 || 53.417 || 103.825 || 2300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7379
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7379
JoR: 1.287 || Eff_3d: 30.083 || Jaccard: 0.726 || Factor: 353.009 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7906
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7906
JoR: 1.265 || Eff_3d: 43.644 || Jaccard: 0.774 || Factor: 368.226 || RMSE_lat: 61.172 || RMSE_ax: 83.474 || RMSE_x: 45.542 || RMSE_y: 40.841 || Cost: 42.320 || Recall: 0.778 || Precision: 0.993 || BatchNr.: 3600 || Cost: 42.320 || Time Upd.: 386.3 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 43.644 || 61.172 || 83.474 || 3600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8460
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8460
JoR: 1.311 || Eff_3d: 36.711 || Jaccard: 0.812 || Factor: 388.721 || RMSE_lat: 61.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8231
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8231
JoR: 1.386 || Eff_3d: 45.518 || Jaccard: 0.797 || Factor: 374.665 || RMSE_lat: 57.519 || RMSE_ax: 86.969 || RMSE_x: 41.369 || RMSE_y: 39.963 || Cost: 38.613 || Recall: 0.805 || Precision: 0.988 || BatchNr.: 4900 || Cost: 38.613 || Time Upd.: 372.9 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 45.518 || 57.519 || 86.969 || 4900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8337
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8337
JoR: 1.382 || Eff_3d: 53.429 || Jaccard: 0.798 || Factor: 364.064 || RMSE_lat: 57.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7737
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7737
JoR: 1.401 || Eff_3d: 54.324 || Jaccard: 0.762 || Factor: 336.861 || RMSE_lat: 54.410 || RMSE_ax: 42.716 || RMSE_x: 39.869 || RMSE_y: 37.027 || Cost: 35.442 || Recall: 0.764 || Precision: 0.997 || BatchNr.: 6200 || Cost: 35.442 || Time Upd.: 395.6 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.324 || 54.410 || 42.716 || 6200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7647
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7647
JoR: 1.395 || Eff_3d: 54.265 || Jaccard: 0.757 || Factor: 325.370 || RMSE_lat: 54.

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8080
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8080
JoR: 1.484 || Eff_3d: 57.448 || Jaccard: 0.800 || Factor: 324.013 || RMSE_lat: 53.930 || RMSE_ax: 38.036 || RMSE_x: 39.343 || RMSE_y: 36.886 || Cost: 30.554 || Recall: 0.800 || Precision: 1.000 || BatchNr.: 14000 || Cost: 30.554 || Time Upd.: 381.8 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 57.448 || 53.930 || 38.036 || 14000


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8059
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8059
JoR: 1.478 || Eff_3d: 57.657 || Jaccard: 0.798 || Factor: 333.008 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7840
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7840
JoR: 1.466 || Eff_3d: 57.087 || Jaccard: 0.776 || Factor: 327.094 || RMSE_lat: 52.944 || RMSE_ax: 34.848 || RMSE_x: 37.889 || RMSE_y: 36.979 || Cost: 30.397 || Recall: 0.777 || Precision: 1.000 || BatchNr.: 15300 || Cost: 30.397 || Time Upd.: 377.1 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 57.087 || 52.944 || 34.848 || 15300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8073
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8073
JoR: 1.474 || Eff_3d: 57.778 || Jaccard: 0.799 || Factor: 337.270 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8262
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8262
JoR: 1.501 || Eff_3d: 58.210 || Jaccard: 0.817 || Factor: 355.354 || RMSE_lat: 54.434 || RMSE_ax: 37.386 || RMSE_x: 39.415 || RMSE_y: 37.543 || Cost: 29.587 || Recall: 0.818 || Precision: 0.999 || BatchNr.: 16600 || Cost: 29.587 || Time Upd.: 376.4 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.210 || 54.434 || 37.386 || 16600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:7451
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 7451
JoR: 1.429 || Eff_3d: 55.139 || Jaccard: 0.738 || Factor: 306.933 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8178
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8178
JoR: 1.504 || Eff_3d: 58.030 || Jaccard: 0.810 || Factor: 329.811 || RMSE_lat: 53.870 || RMSE_ax: 37.853 || RMSE_x: 38.865 || RMSE_y: 37.302 || Cost: 29.715 || Recall: 0.810 || Precision: 1.000 || BatchNr.: 17900 || Cost: 29.715 || Time Upd.: 441.9 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.030 || 53.870 || 37.853 || 17900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8143
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8143
JoR: 1.487 || Eff_3d: 57.641 || Jaccard: 0.806 || Factor: 332.373 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8185
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8185
JoR: 1.516 || Eff_3d: 58.702 || Jaccard: 0.810 || Factor: 340.729 || RMSE_lat: 53.451 || RMSE_ax: 35.219 || RMSE_x: 38.354 || RMSE_y: 37.229 || Cost: 30.012 || Recall: 0.811 || Precision: 1.000 || BatchNr.: 19200 || Cost: 30.012 || Time Upd.: 404.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.702 || 53.451 || 35.219 || 19200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10094, preds:8388
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10094 ,preds: 8388
JoR: 1.510 || Eff_3d: 58.882 || Jaccard: 0.830 || Factor: 343.549 || RMSE_lat: 5